# Python Script to Generate Cropped Images for CNN Classification Training

#### Cell to Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


#### Importing Libraries

In [ ]:
import os
import cv2
from decimal import Decimal
from PIL import Image 
import PIL 
import numpy as np
from natsort import natsorted 
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Reading File Names

In [ ]:
txt_list_0 = []
image_list_0 = []


for i in os.listdir("drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/"):
  s = "drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/"
  if ".txt" in i:
    s += i
    txt_list_0.append(s)
  elif ".jpeg" in i:
    s += i
    image_list_0.append(s) 
  
image_list_0 = natsorted(image_list_0)
txt_list_0 = natsorted(txt_list_0)
print(txt_list_0)

['drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/1.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/2.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/3.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/4.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/5.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/6.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/7.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/8.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/9.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/10.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/11.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/12.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/obj/13.txt', 'drive/MyDrive/oxwash/dataset2/data/individual_care_symbols

In [ ]:
txt_list_1 = []
image_list_1 = []

for i in os.listdir("drive/MyDrive/oxwash/dataset1/final_v2/"):
  s = "drive/MyDrive/oxwash/dataset1/final_v2/"
  if ".txt" in i:
    s += i
    txt_list_1.append(s)
  elif ".jpg" in i:
    s += i
    image_list_1.append(s) 
  
image_list_1 = natsorted(image_list_1)
txt_list_1 = natsorted(txt_list_1)
print(len(image_list_1),len(txt_list_1))
print(txt_list_1[131])
print(image_list_1[131])

199 199
drive/MyDrive/oxwash/dataset1/final_v2/133.txt
drive/MyDrive/oxwash/dataset1/final_v2/133.jpg


Creating Directory for Images of Bounding Boxes

In [ ]:
#os.mkdir("drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/boundingboxes")
categories = ["unspecified_material","bleaching_any",
              "bleaching_not",
              "bleaching_onlySome","drying_flat","drying_flat_drip","drying_flat_drip_shade","drying_flat_shade","drying_line",
              "drying_line_drip","drying_line_drip_shade","drying_line_shade","drying_tumble_lower_60","drying_tumble_normal_80","drying_tumble_not",
              "ironing_not","ironing_plate_110","ironing_plate_150","ironing_plate_200","professional_dry_mild_F","professional_dry_mild_P",
              "professional_dry_normal_F","professional_dry_normal_P","professional_dry_not","professional_wet_mild_W","professional_wet_normal_W",
              "professional_wet_not","professional_wet_veryMild_W","washing_hand_40","washing_mild_30","washing_mild_40","washing_mild_60",
              "washing_normal_30","washing_normal_40","washing_normal_60","washing_normal_95","washing_not","washing_veryMild_30","washing_veryMild_40"]

os.mkdir("drive/MyDrive/oxwash/dataset_v5")
for label in categories:
   os.mkdir("drive/MyDrive/oxwash/dataset_v5/" + str(label))


Displaying Bounding Boxes in the image

Calculating x,y,w,h 

In [ ]:
def coordinates(lines,img):
  crop_list = []
  dh,dw,_ = img.shape
 
  for i in range(len(lines)):
    _, x, y, w, h = map(float, lines[i].split(' '))

  
    left = int((x - w / 2) * dw)
    width = int((x + w / 2) * dw) # width
    top = int((y - h / 2) * dh)
    height = int((y + h / 2) * dh) #height

    if left < 0:
      left = 0
    if width > dw - 1:
      width = dw - 1
    if top < 0:
      top = 0
    if height > dh - 1:
      height = dh - 1   

    # cv2.rectangle(img, (left, top), (width, height), (0, 0, 255), 1)
    ROI= img[top: height,  left: width ]

 

    crop_list.append(ROI)

    
  return img,crop_list

In [ ]:
label_dir_0 = "drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/Annotated_Revised.csv"
label_dir_1 = "drive/MyDrive/oxwash/dataset1/labeled_photos_revised_v2.csv"

label_df_0 = pd.read_csv(label_dir_0,header = None)
label_df_1 = pd.read_csv(label_dir_1,header = None)

label_df_1.head(n=5)

,0,1,2,3,4,5
0,washing_hand_40,bleaching_not,ironing_plate_110,professional_dry_mild_P,drying_flat,NaN
1,washing_mild_30,bleaching_not,ironing_plate_110,drying_tumble_not,professional_dry_normal_P,NaN
2,washing_mild_30,bleaching_not,drying_tumble_not,ironing_plate_110,professional_dry_not,NaN
3,washing_mild_30,bleaching_not,ironing_plate_110,drying_flat,professional_dry_normal_P,NaN
4,washing_hand_40,bleaching_not,ironing_plate_110,professional_dry_normal_P,drying_flat,NaN


In [ ]:
temp_row = label_df_0.iloc[0]
print(str(temp_row.to_list()))
print(len(temp_row.to_list()))
print(label_df_0.shape[0])

['washing_normal_30', 'bleaching_not', 'ironing_plate_150', 'professional_dry_normal_P', 'drying_flat', nan]
6
40


In [ ]:
label_df_1 = label_df_1.iloc[0:199]
temp_row = label_df_1.iloc[198]
print(str(temp_row.to_list()))
print(len(temp_row.to_list()))
print(label_df_1.shape[0])

['washing_not', 'bleaching_not', 'drying_tumble_not', 'ironing_plate_150', 'professional_dry_normal_P', nan]
6
199


In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x

In [ ]:
gen = ImageDataGenerator(rescale = 1./255,rotation_range=45,shear_range=0.25,channel_shift_range=10, 
horizontal_flip=False, brightness_range=[0.1,3.0],fill_mode='reflect',preprocessing_function= (lambda x:random_invert_img(x, 0.5)))

In [ ]:
count = 0

In [ ]:
for i in range(label_df_0.shape[0]):
  temp_row_0 = label_df_0.iloc[i]
  temp_row_list_0 = temp_row_0.tolist()
  with open(txt_list_0[i]) as f:
      lines_0 = f.readlines()
    
  img_0 , crop_list_0 = coordinates(lines_0,np.array(Image.open(image_list_0[i])))

  for j in range(len(temp_row_list_0)):
    if str(temp_row_list_0[j]) == "nan":
      continue

    image = np.expand_dims(crop_list_0[j],0)*255
  
    path = "drive/MyDrive/oxwash/dataset_v5/"+str(temp_row_list_0[j]) + "/" 
    aug_iter = gen.flow(image, save_to_dir=path, save_prefix=str(count),save_format='png')
    aug_images = [next(aug_iter)[0].astype('float32') for i2 in range(12)]
    print(path+str(count))
    count += 1
    
  #data = Image.fromarray(img)
  #data.save("drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/boundingboxes/bbox"+ str(i) + ".png")

drive/MyDrive/oxwash/dataset_v5/washing_normal_30/0
drive/MyDrive/oxwash/dataset_v5/bleaching_not/1
drive/MyDrive/oxwash/dataset_v5/ironing_plate_150/2
drive/MyDrive/oxwash/dataset_v5/professional_dry_normal_P/3
drive/MyDrive/oxwash/dataset_v5/drying_flat/4
drive/MyDrive/oxwash/dataset_v5/washing_normal_40/5
drive/MyDrive/oxwash/dataset_v5/bleaching_not/6
drive/MyDrive/oxwash/dataset_v5/drying_tumble_lower_60/7
drive/MyDrive/oxwash/dataset_v5/ironing_plate_150/8
drive/MyDrive/oxwash/dataset_v5/professional_dry_not/9
drive/MyDrive/oxwash/dataset_v5/washing_normal_30/10
drive/MyDrive/oxwash/dataset_v5/bleaching_not/11
drive/MyDrive/oxwash/dataset_v5/ironing_plate_110/12
drive/MyDrive/oxwash/dataset_v5/professional_dry_normal_P/13
drive/MyDrive/oxwash/dataset_v5/drying_flat/14
drive/MyDrive/oxwash/dataset_v5/washing_normal_30/15
drive/MyDrive/oxwash/dataset_v5/bleaching_not/16
drive/MyDrive/oxwash/dataset_v5/ironing_plate_150/17
drive/MyDrive/oxwash/dataset_v5/professional_dry_normal_P/18

In [ ]:
for i in range(199):
  temp_row_1 = label_df_1.iloc[i]
  temp_row_list_1 = temp_row_1.tolist()
  with open(txt_list_1[i]) as f:
      lines_1 = f.readlines()
  #print(image_list_1[i],txt_list_1[i])  
  img_1 , crop_list_1 = coordinates(lines_1,np.array(Image.open(image_list_1[i])))  

  for j in range(len(temp_row_list_1)):
    if str(temp_row_list_1[j]) == "nan":
      continue
    
    image = np.expand_dims(crop_list_1[j],0)*255
  
    path = "drive/MyDrive/oxwash/dataset_v5/"+str(temp_row_list_1[j]) + "/" 
    aug_iter = gen.flow(image, save_to_dir=path, save_prefix=str(count),save_format='png')
    aug_images = [next(aug_iter)[0].astype('float32') for i2 in range(12)]
    print(path+str(count))
    count += 1

    ''' data = Image.fromarray(crop_list_1[j])
    path = "drive/MyDrive/oxwash/dataset_v5/"+str(temp_row_list_1[j]) + "/" + str(count) + ".png"
    data.save(path)
    print(path)
    count += 1 '''


drive/MyDrive/oxwash/dataset_v5/washing_hand_40/0
drive/MyDrive/oxwash/dataset_v5/bleaching_not/1
drive/MyDrive/oxwash/dataset_v5/ironing_plate_110/2
drive/MyDrive/oxwash/dataset_v5/professional_dry_mild_P/3
drive/MyDrive/oxwash/dataset_v5/drying_flat/4
drive/MyDrive/oxwash/dataset_v5/washing_mild_30/5
drive/MyDrive/oxwash/dataset_v5/bleaching_not/6
drive/MyDrive/oxwash/dataset_v5/ironing_plate_110/7
drive/MyDrive/oxwash/dataset_v5/drying_tumble_not/8
drive/MyDrive/oxwash/dataset_v5/professional_dry_normal_P/9
drive/MyDrive/oxwash/dataset_v5/washing_mild_30/10
drive/MyDrive/oxwash/dataset_v5/bleaching_not/11
drive/MyDrive/oxwash/dataset_v5/drying_tumble_not/12
drive/MyDrive/oxwash/dataset_v5/ironing_plate_110/13
drive/MyDrive/oxwash/dataset_v5/professional_dry_not/14
drive/MyDrive/oxwash/dataset_v5/washing_mild_30/15
drive/MyDrive/oxwash/dataset_v5/bleaching_not/16
drive/MyDrive/oxwash/dataset_v5/ironing_plate_110/17
drive/MyDrive/oxwash/dataset_v5/drying_flat/18
drive/MyDrive/oxwash/d

In [ ]:
#!rm -r drive/MyDrive/oxwash/dataset2/data/individual_care_symbols/boundingboxes
!rm -r drive/MyDrive/oxwash/dataset_v5 

rm: cannot remove 'drive/MyDrive/oxwash/dataset_v5': No such file or directory
